In [1]:
import pandas as pd
import numpy as np
from glob import glob
import re

In [2]:
ohco = ['speech_id','speaker','para_id','sent_id','token_id']

In [3]:
df = pd.read_json('data/ucsb_speeches_2016.json')

In [4]:
df.head()

,link,title,date,person,transcript
0,/documents/remarks-town-hall-meeting-portsmout...,"Remarks at a Town Hall Meeting in Portsmouth, ...",2015-12-29 00:00:00+00:00,Hillary Clinton,\nCLINTON: Wow. Thank you. Thank you all. Than...
1,/documents/remarks-the-university-minnesota-mi...,Remarks at the University of Minnesota in Minn...,2015-12-15 00:00:00+00:00,Hillary Clinton,\nThank you. Thank you all very much. Thank yo...
2,/documents/interview-with-george-stephanopoulo...,Interview with George Stephanopoulos of ABC Ne...,2015-12-06 00:00:00+00:00,Hillary Clinton,\nSTEPHANOPOULOS: And we'll hear more on that ...
3,/documents/interview-with-charlie-rose,Interview with Charlie Rose,2015-12-01 00:00:00+00:00,Hillary Clinton,"\nROSE: She is a former first lady, a former s..."
4,/documents/remarks-and-question-and-answer-ses...,Remarks and a Question and Answer Session at t...,2015-11-19 00:00:00+00:00,Hillary Clinton,\nCLINTON: Thank you. Thank you very much. [ap...


In [5]:
df.index.name = 'speech_id'
library = df[['link','title','date','person']]

In [6]:
# first OHCO level - split out speakers, using e.g. "CLINTON:"
# set default speaker for each speech - we'll set the specific ones later.
df['speaker'] = [x[1] for x in df['person'].str.upper().str.rsplit(' ',1)]

In [7]:
df = df.reset_index().set_index(['speech_id','speaker'])

In [8]:
# second OHCO level - split out paragraphs, using "\n" as the separator (for these transcripts). 
# Remove initial/trailing whitespace, including \n
df = df['transcript'].str.strip().str.split("\n", expand=True)\
    .stack().to_frame().rename(columns={0:'para_str'})
df.index.names = ohco[0:3]

In [9]:
df = df.reset_index().set_index(['speech_id','para_id'])

In [10]:
# whenever a paragraph starts with a caps name e.g. CLINTON:
# use that as the speaker until the next caps name
df['speaker'] = df['para_str'].str.extract(r'([A-Z]+)(:)')[0].ffill()
# remove the non-Trump/Clinton speakers
df = df[df['speaker'].isin(['TRUMP','CLINTON'])]

In [11]:
df = df.reset_index().set_index(ohco[0:3])

In [12]:
# remove the caps names
df['para_str'] = df['para_str'].str.replace(r'[A-Z]+: ','')

In [13]:
# third ohco level - sentence

# Alvarado used NLTK sentence tokenizer to split sentences. 
# but let's try it the lazy way to start
df = df['para_str'].str.split("[.!?]+", expand=True)\
    .stack().to_frame().rename(columns={0:'sent_str'})
df.index.names = ohco[0:4]

In [14]:
# drop 0-length strings (usually at end of paragraph)
df = df[df['sent_str'].str.len() > 0]

In [15]:
df.head()

sent_str
speech_id speaker para_id sent_id                                                   
0         CLINTON 0       0                                                      Wow
                          1                                                Thank you
                          2                                            Thank you all
                          3                                                Thank you
                          4         I am really delighted to be here on the first...

In [16]:
# fourth ohco level - tokens
import nltk

In [48]:
token = df['sent_str'].apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
    .stack().to_frame()\
    .rename(columns={0:'pos_tuple'})
token['pos'] = token.pos_tuple.apply(lambda x: x[1])
token['token_str'] = token.pos_tuple.apply(lambda x: x[0])
token = token.drop('pos_tuple', 1)

/Users/maxwell/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [49]:
token.index.names = ohco # ok, all done

In [50]:
token.head()

pos token_str
speech_id speaker para_id sent_id token_id               
0         CLINTON 0       0       0          NN       Wow
                          1       0          NN     Thank
                                  1         PRP       you
                          2       0         NNP     Thank
                                  1         PRP       you

In [51]:
# Lowercase, remove non-word characters
token['term_str'] = token['token_str'].str.lower().str.replace('[\W_]', '')
# drop words which consist entirely of non-word characters
token = token[token.term_str!=''].sort_index()

In [52]:
# Checkpoint
token.to_parquet('data/token.parquet')
library.to_parquet('data/library.parquet')

In [53]:
# Load from checkpoint
if 'token' not in locals():
    token = pd.read_parquet('data/token.parquet')
if 'library' not in locals():
    library = pd.read_parquet('data/library.parquet')

In [54]:
# create vocab table
# 
vocab = token.term_str.value_counts().to_frame().rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
vocab.index.name = 'term_id'

In [55]:
# add stopwords
vocab['stop'] = 0
vocab.loc[vocab['term_str'].isin(nltk.corpus.stopwords.words('english')),'stop'] = 1

In [56]:
# (optional) add stems
#from nltk.stem.porter import PorterStemmer
#stemmer = PorterStemmer()
#vocab['p_stem'] = vocab.term_str.apply(stemmer.stem)

In [57]:
# add term rank 
if 'term_rank' not in vocab.columns:
    vocab = vocab.sort_values('n', ascending=False).reset_index()
    vocab.index.name = 'term_rank'
    vocab = vocab.reset_index().set_index('term_id')
    vocab['term_rank'] = vocab['term_rank'] + 1 # start with 1 instead of 0

In [58]:
vocab.sort_values('n', ascending=False).head()

,term_rank,term_str,n,stop
term_id,,,,
10832,1,the,13552,1
11006,2,to,12704,1
782,3,and,12178,1
5499,4,i,7337,1
7517,5,of,6918,1


In [59]:
# add term id back to token table, for easy joining
token['term_id'] = token.term_str.map(vocab.reset_index().set_index('term_str').term_id)

In [60]:
# Checkpoint
vocab.to_parquet('data/vocab.parquet')

## Create DT matrices 
(Document-Term matrix, using Bag of Words and TF-IDF)

One each for Trump and Clinton

In [61]:
# Load from checkpoint
if 'vocab' not in locals():
    vocab = pd.read_parquet('data/vocab.parquet')

In [62]:
bag = ohco[:1] # bag size = 1 speech

In [63]:
# identify stopwords
token['stop'] = 0
token.loc[token['term_str'].isin(nltk.corpus.stopwords.words('english')),'stop'] = 1

In [64]:
#remove stopwords
token=token[token.stop==0]

In [65]:
# 100 speeches, take 20% for test set
test_ids = np.random.choice(token.reset_index().speech_id.unique(),size=20,replace=False)

In [66]:
token = token.reset_index().set_index('speech_id')

In [67]:
test_token = token.loc[test_ids]
train_token = token.loc[~token.index.isin(test_ids)]

In [68]:
# Split into Trump/Clinton
clinton_train = train_token.query('speaker=="CLINTON"')
trump_train = train_token.query('speaker=="TRUMP"')

In [69]:
clinton_count = len(clinton_train.index.unique())
trump_count = len(trump_train.index.unique())
total_count = clinton_count + trump_count
priors = [clinton_count/total_count,trump_count/total_count]
priors

[0.48148148148148145, 0.5185185185185185]

In [70]:
def create_vocab(token, a=1):
    vocab = token.term_str.value_counts().to_frame().rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
    vocab.index.name = 'term_id'
    vocab['stop'] = 0
    vocab.loc[vocab['term_str'].isin(nltk.corpus.stopwords.words('english')),'stop'] = 1
    if 'term_rank' not in vocab.columns:
        vocab = vocab.sort_values('n', ascending=False).reset_index()
        vocab.index.name = 'term_rank'
        vocab = vocab.reset_index().set_index('term_id')
        vocab['term_rank'] = vocab['term_rank'] + 1 # start with 1 instead of 0
    token['term_id'] = token.term_str.map(vocab.reset_index().set_index('term_str').term_id)
    BOW = token.groupby(bag+['term_id']).term_id.count()\
    .to_frame().rename(columns={'term_id':'n'})
    DTCM = BOW['n'].unstack().fillna(0).astype('int')
    # Term frequency - normalized
    TF = (DTCM.T / DTCM.T.sum()).T
    DF = DTCM[DTCM > 0].count()
    freq=TF.sum().reset_index()
    #freq=DTCM.sum().reset_index() #optional to get more variance in the likelihood
    vocab['freq'] = freq[0]
    tf = vocab.freq # term frequency (normalized)
    N = vocab.freq.sum() # sum of all term frequencies in this class
    V = vocab.shape[0] # vocab size
    # https://stackoverflow.com/questions/3704570/in-python-small-floats-tending-to-zero
    # https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html
    # using the log likelihood for these reasons
    vocab['likelihood'] = np.log((tf+a) / (N+ a*V))
    vocab.sort_values('likelihood',ascending=False)
    return vocab

In [71]:
vocab_c = create_vocab(clinton_train).reset_index().set_index('term_str')
vocab_t = create_vocab(trump_train).reset_index().set_index('term_str')

/Users/maxwell/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [77]:
def NB_classifier(token_test, class1_vocab, class2_vocab, class_1_name = 'Class 1', class_2_name = 'Class 2', priors=[.5,.5]):
    d = {'speech_id': token_test.index.unique(), 'true_value': "", 'prediction': ""}
    results = pd.DataFrame(d).set_index('speech_id')
    
    for i in results.index:
        results['true_value'].loc[i] = test_token.at[i,'speaker'][0]
        
        speech = token_test.loc[i]
        
        speech = speech.merge(class1_vocab['likelihood'], how='left', on="term_str")\
        .rename(columns={'likelihood': class_1_name})
        speech = speech.merge(class2_vocab['likelihood'], how='left', on="term_str")\
        .rename(columns={'likelihood': class_2_name})
        
        predict = pd.DataFrame(speech[[class_1_name,class_2_name]].sum())
        logpriors = np.log(priors)
        predict[0] = predict[0] + logpriors
        predict = predict.sort_values(by=0, ascending=False)
        classification = predict.index[0]
        
        results['prediction'].loc[i] = classification
        
    return results

In [78]:
# class 1 is clinton, class 2 is trump

results=NB_classifier(test_token,vocab_c,vocab_t, class_1_name = 'CLINTON', class_2_name = 'TRUMP', priors=priors)
results

,true_value,prediction
speech_id,,
139,TRUMP,CLINTON
39,TRUMP,CLINTON
12,CLINTON,CLINTON
40,TRUMP,CLINTON
32,TRUMP,CLINTON
11,CLINTON,TRUMP
162,CLINTON,TRUMP
31,CLINTON,CLINTON
137,TRUMP,CLINTON


In [79]:
from sklearn.metrics import confusion_matrix
confusion_matrix(results.true_value, results.prediction)

array([[ 4,  6],
       [10,  0]])

In [136]:
speech = test_token.loc[44]
        
speech = speech.merge(vocab_c['likelihood'], how='left', on="term_str")\
        .rename(columns={'likelihood': 'clinton'})
speech = speech.merge(vocab_t['likelihood'], how='left', on="term_str")\
        .rename(columns={'likelihood': 'trump'})
        
predict = pd.DataFrame(speech[['clinton','trump']].sum())
logpriors = np.log(priors)
predict[0] = predict[0] + logpriors
predict = predict.sort_values(by=0, ascending=False)

In [137]:
predict

,0
clinton,-13730.913300
trump,-13919.136719
